<a href="https://colab.research.google.com/github/taxevader007/markovModelUni/blob/main/proyecto_grupo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matematicas Aplicadas II


# **Información del grupo**

1. Santiago Espinosa Giraldo A00399531
2. Haider Valencia Zuniga A00400766
3. Sofia Romero A00396446
4. Valentina Arana A00399932


# Descripción:

 Modelo de prediccion del crudo del petroleo usando cadenas de markov y regresion lineal




# Implementación numérica:

Utilizar un métodos numéricos

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

ticker = "UKOG.L" #petroleo
data = yf.download(ticker, start= "2020-01-01",end= "2024-04-01")


[*********************100%%**********************]  1 of 1 completed


In [ ]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,0.850,0.880,0.821,0.850,0.850,29215102
2020-01-03,0.850,0.864,0.785,0.825,0.825,51546289
2020-01-06,0.825,0.823,0.775,0.800,0.800,42355228
2020-01-07,0.800,0.850,0.770,0.800,0.800,43720619
2020-01-08,0.800,0.900,0.750,0.800,0.800,49945463
...,...,...,...,...,...,...
2024-03-22,0.058,0.065,0.050,0.058,0.058,13029447
2024-03-25,0.058,0.064,0.050,0.058,0.058,9656422
2024-03-26,0.058,0.064,0.050,0.058,0.058,8186379


se cargan los datos de el activo en ese periodo de tiempo

In [ ]:
data["daily_return"] = data["Adj Close"].pct_change()
#percentage difference yesterday and today

data["state"] = np.where(data["daily_return"] >= 0, "up" , "down")
#where the daily return is up or down

data

,Open,High,Low,Close,Adj Close,Volume,daily_return,state
Date,,,,,,,,
2020-01-02,0.850,0.880,0.821,0.850,0.850,29215102,NaN,down
2020-01-03,0.850,0.864,0.785,0.825,0.825,51546289,-0.029412,down
2020-01-06,0.825,0.823,0.775,0.800,0.800,42355228,-0.030303,down
2020-01-07,0.800,0.850,0.770,0.800,0.800,43720619,0.000000,up
2020-01-08,0.800,0.900,0.750,0.800,0.800,49945463,0.000000,up
...,...,...,...,...,...,...,...,...
2024-03-22,0.058,0.065,0.050,0.058,0.058,13029447,0.000000,up
2024-03-25,0.058,0.064,0.050,0.058,0.058,9656422,0.000000,up
2024-03-26,0.058,0.064,0.050,0.058,0.058,8186379,0.000000,up


datos de todos los dias con apertura y cierre y si cerro arriba o abajo con respecto al dia anterior

In [ ]:
# Calculate counts
up_counts = len(data[data["state"] == "up"])
down_counts = len(data[data["state"] == "down"])

# Calculate transition probabilities
up_to_up = len(data[(data["state"] == "up") & (data["state"].shift(-1) == "up")]) / len(data.query('state == "up"'))
up_to_down = len(data[(data["state"] == "up") & (data["state"].shift(-1) == "down")]) / len(data.query('state == "up"'))
down_to_up = len(data[(data["state"] == "down") & (data["state"].shift(-1) == "up")]) / len(data.query('state == "down"'))
down_to_down = len(data[(data["state"] == "down") & (data["state"].shift(-1) == "down")]) / len(data.query('state == "down"'))

# Create transition matrix
transition_matrix = pd.DataFrame({
    "up": [up_to_up, up_to_down],
    "down": [down_to_up, down_to_down]
}, index=["up", "down"])

print(transition_matrix)

            up      down
up    0.503356  0.624473
down  0.494966  0.375527


The matrix represents the transition probabilities between two states: “up” and “down.”


The rows and columns correspond to the current state and the next state, respectively.


Here’s the breakdown:
Up to Up (0.503356): This means that if the current state is “up,” there is a 50.34% chance that it will remain in the “up” state in the next step.

Up to Down (0.624473): If the current state is “up,” there is a 62.45% chance that it will transition to the “down” state.

Down to Up (0.494966): If the current state is “down,” there is a 49.50% chance that it will transition to the “up” state.

Down to Down (0.375527): This means that if the current state is “down,” there is a 37.55% chance that it will remain in the “down” state in the next step.

In summary, the transition matrix provides insights into how likely the system is to change from one state to another. In this case, the probabilities suggest that transitions from “up” to “down” are more likely than vice versa.

In [ ]:
prob_stay_up = transition_matrix.loc["up", "up"]
prob_stay_down = transition_matrix.loc["down", "down"]

print(f"Probability of staying 'up': {prob_stay_up:.4f}")
print(f"Probability of staying 'down': {prob_stay_down:.4f}")

Probability of staying 'up': 0.5034
Probability of staying 'down': 0.3755


Up to Up (0.503356):
If the current state is “up,” there is a 50.34% chance that it will remain in the “up” state in the next step.

Down to Down (0.375527):
If the current state is “down,” there is a 37.55% chance that it will remain in the “down” state in the next step.

In [ ]:
down_after_five_up = len(data[(data["state"] == "up") & (data["state"].shift(-1) == "down") & (data["state"].shift(-2) == "down") & (data["state"].shift(-3) == "down") ]) / len(data.query('state == "up"'))

print(f"The probability of a down day after 6 consecutive up days is approximately {down_after_five_up:.4f}")

The probability of a down day after 6 consecutive up days is approximately 0.0688
